In [5]:
import ee
import pandas as pd
import os
import time

# ee.Authenticate()

# Initialize Earth Engine
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
    print('Google Earth Engine initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine failed to initialize!', e)
    raise

Google Earth Engine initialized successfully!


In [7]:
sorted(os.listdir("./shc_data/NORMALISED_DATA/2024"))

['.ipynb_checkpoints',
 'ANDAMAN & NICOBAR_2024.csv',
 'ANDHRA PRADESH_2024.csv',
 'ARUNACHAL PRADESH_2024.csv',
 'ASSAM_2024.csv',
 'BIHAR_2024.csv',
 'CHHATTISGARH_2024.csv',
 'GOA_2024.csv',
 'GUJARAT_2024.csv',
 'HARYANA_2024.csv',
 'HIMACHAL PRADESH_2024.csv',
 'JAMMU & KASHMIR_2024.csv',
 'JHARKHAND_2024.csv',
 'KARNATAKA_2024.csv',
 'LADAKH_2024.csv',
 'MADHYA PRADESH_2024.csv',
 'MAHARASHTRA_2024.csv',
 'MEGHALAYA_2024.csv',
 'MIZORAM_2024.csv',
 'NAGALAND_2024.csv',
 'ODISHA_2024.csv',
 'PUDUCHERRY_2024.csv',
 'PUNJAB_2024.csv',
 'RAJASTHAN_2024.csv',
 'SIKKIM_2024.csv',
 'TAMIL NADU_2024.csv',
 'TELANGANA_2024.csv',
 'TRIPURA_2024.csv',
 'UTTAR PRADESH_2024.csv',
 'UTTARAKHAND_2024.csv',
 'WEST BENGAL_2024.csv']

0: 
Andaman & Nicobar
1: 
Andhra Pradesh
2: 
Arunachal Pradesh
3: 
Assam
4: 
Bihar
5: 
Chandigarh
6: 
Chhattishgarh
7: 
Daman and Diu and Dadra and Nagar Haveli
8: 
Delhi
9: 
Goa
10: 
Gujarat
11: 
Haryana
12: 
Himachal Pradesh
13: 
Jammu and Kashmir
14: 
Jharkhand
15: 
Karnataka
16: 
Kerala
17: 
Ladakh
18: 
Lakshadweep
19: 
Madhya Pradesh
20: 
Maharashtra
21: 
Manipur
22: 
Meghalaya
23: 
Mizoram
24: 
Nagaland
25: 
Odisha
26: 
Puducherry
27: 
Puducherry
28: 
Punjab
29: 
Rajasthan
30: 
Sikkim
31: 
Tamilnadu
32: 
Telengana
33: 
Tripura
34: 
Uttar Pradesh
35: 
Uttarakhand
36: 
West Bengal

In [41]:
# === CONFIGURABLE PARAMETERS ===
DATA_DIR = "./shc_data/NORMALISED_DATA/2024"
INPUT_CSV = os.path.join(DATA_DIR, "BIHAR_2024.csv")  # CSV containing soil properties
# OUTPUT_FOLDER = "./experiments/LA_23_24"  # Folder to save results
BATCH_SIZE = 3000  # Number of points processed per batch

india_districts = ee.FeatureCollection("projects/ee-aakash312000/assets/state")
state = india_districts.filter(ee.Filter.eq('State_Name', 'Bihar'))

# Create output folder if not exists
# os.makedirs(OUTPUT_FOLDER, exist_ok=True)
collections = None
SD = "2024-01-01"
ED = "2024-12-31"

In [32]:
import math
import geemap
import datetime

def renameBands(image):
    return image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'], ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])

def maskL8clouds(image):
    qa = image.select('QA_PIXEL')
    cloudMask = qa.bitwiseAnd(1 << 4).eq(0)
    shadowMask = qa.bitwiseAnd(1 << 3).eq(0)
    mask = cloudMask.And(shadowMask)
    scaled = image.multiply(0.0000275).add(-0.2)
    bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
    return scaled.updateMask(mask).select(bands, ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']).toFloat()

def maskS2clouds(image):
    qa = image.select('QA60');
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    scaled = image.divide(10000)
    scaled = scaled.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])
    return scaled.updateMask(mask).toFloat()

# Apply harmonization for Sentinel-2
slopes = [1.0946, 1.0043, 1.0524, 0.8954, 1.0049, 1.0002]
intercepts = [-0.0107, 0.0026, -0.0015, 0.0033, 0.0065, 0.0046]

def convert(image):
  return image.select(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']).multiply(slopes).add(intercepts)

# date_range_month = {
#     "Jan" : (ee.Date("2023-01-01"), ee.Date("2023-01-31")),
#     "Feb" : (ee.Date("2023-02-01"), ee.Date("2023-02-28")),
#     "Mar" : (ee.Date("2023-03-01"), ee.Date("2023-03-31")),
#     "Apr" : (ee.Date("2023-04-01"), ee.Date("2023-04-30")),
#     "May" : (ee.Date("2023-05-01"), ee.Date("2023-05-31")),
#     "Jun" : (ee.Date("2023-06-01"), ee.Date("2023-06-30")),
#     "Jul" : (ee.Date("2023-07-01"), ee.Date("2023-07-31")),
#     "Aug" : (ee.Date("2023-08-01"), ee.Date("2023-08-31")),
#     "Sep" : (ee.Date("2023-09-01"), ee.Date("2023-09-30")),
#     "Oct" : (ee.Date("2023-10-01"), ee.Date("2023-10-31")),
#     "Nov" : (ee.Date("2023-11-01"), ee.Date("2023-11-30")),
#     "Dec" : (ee.Date("2023-12-01"), ee.Date("2023-12-31"))
# }

def harmonize_collections(start_date, end_date, roi, month):
    """Fetches harmonized Sentinel-2 and Landsat collections with indices."""

    def update(image):
      return image.multiply(0.002)

    S2 = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
        .map(maskS2clouds)
    )
    
    temp = (
            ee.ImageCollection("MODIS/061/MOD11A2")
            .select('LST_Day_1km')
            .filterDate(start_date, end_date)
            .filterBounds(roi)
            .map(lambda img: update(img))
            .mean()
            .rename('temp')
    )
    
    preci = (
            ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
            .filterDate(start_date, end_date)
            .filterBounds(roi)
            .mean()
            .rename('precipitation')      
    )

    elevation = ee.Image("USGS/SRTMGL1_003").select("elevation").clip(roi)
    slope = ee.Terrain.slope(elevation).rename('slope')
    aspect = ee.Terrain.aspect(elevation).rename('aspect')
    
    slope_radians = slope.multiply(math.pi).divide(180)
    
    flow_accumulation = ee.Image("MERIT/Hydro/v1_0_1").select("upa").clip(roi) # Upstream Area (Flow Accumulation)
    
    tan_slope = slope_radians.tan()
    safe_slope = tan_slope.where(tan_slope.eq(0), 0.001)
    
    twi = flow_accumulation.divide(safe_slope).log().rename('TWI')

    sand = ee.Image("projects/soilgrids-isric/sand_mean").select(["sand_0-5cm_mean", "sand_5-15cm_mean"]).clip(roi).rename(['sand05', 'sand515']);
    silt = ee.Image("projects/soilgrids-isric/silt_mean").select(["silt_0-5cm_mean", "silt_5-15cm_mean"]).clip(roi).rename(['silt05', 'silt515']);
    clay = ee.Image("projects/soilgrids-isric/clay_mean").select(["clay_0-5cm_mean", "clay_5-15cm_mean"]).clip(roi).rename(['clay05', 'clay515']);

    S2 = S2.map(lambda img: img.toFloat())
    s2 = S2.mean()

    monthly_stack = s2.addBands(temp).addBands(preci)
    if month is not None:
        month_prefix = ee.String(month.lower() + "_")
        renamed_bands = monthly_stack.bandNames().map(lambda b: month_prefix.cat(ee.String(b)))
        monthly_stack = monthly_stack.rename(renamed_bands)
    collection = ee.Image([monthly_stack, elevation, slope, aspect, twi, sand, clay, silt])
    
    return collection

def fetch_indices_for_district(df, district, month, start_date, end_date):
    global collections
    """Processes a district's data in batches and fetches satellite indices."""
    
    # Filter district data
    district_df = df[df["district"] == district].reset_index(drop=True)
    total_features = len(district_df)
    
    processed = 0
    batch_number = 1
    start_time = time.time()
    all_results = []

    while processed < total_features:
        batch_df = district_df.iloc[processed : processed + BATCH_SIZE]
        points = ee.FeatureCollection([
            ee.Feature(ee.Geometry.Point(row['long'], row['lat']), {
                'district' : row['district'],
                'village' : row['village'],
                'date': row['date'],
                'start_date': SD,
                'end_date': ED,
                'N': row['N'],
                'P': row['P'],
                'K': row['K'],
                'B': row['B'],
                'Fe': row['Fe'],
                'Zn': row['Zn'],
                'Cu': row['Cu'],
                'S': row['S'],
                'OC': row['OC'],
                'pH': row['pH'],
                'Mn': row['Mn'],
                'EC': row['EC']
            })
            for _, row in batch_df.iterrows()
        ])


        points = points.filterBounds(state.geometry())
        roi = points.geometry().bounds()
        collections = harmonize_collections(start_date, end_date, roi, month)
        sampled_points = collections.sampleRegions(
            collection=points, scale=30, tileScale=8, geometries=True
        )

        state_name = INPUT_CSV.split("/")[-1].split(".csv")[0].replace(" ", "_").replace("&", "_")

        task = ee.batch.Export.table.toDrive(
            collection=sampled_points,
            description=f"{state_name}_sampled_batch_{batch_number}",
            folder='GEE_Exports',
            fileNamePrefix=f"{state_name}_{district}_batch{batch_number}",
            fileFormat='CSV'
        )
        task.start()

        processed += BATCH_SIZE
        batch_number += 1
        print(f"Batch {batch_number-1} processed. ")

In [33]:
def main():
    global collections
    """Main function to process all districts."""
    df = pd.read_csv(INPUT_CSV)
    
    # Ensure necessary columns exist
    required_columns = {"long", "lat", "district", "village", "date"}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"CSV file must contain columns: {required_columns}")

    # Process districts
    unique_districts = df["district"].unique()
    print(f"Total Districts: {len(unique_districts)}")

    start_date, end_date = ee.Date(SD), ee.Date(ED)
    
    for district in unique_districts:
        start_time = time.perf_counter()

        print(f"\nProcessing District: {district}")
        batch_df = df[df["district"] == district].reset_index(drop=True)
        total_features = len(batch_df)
        print(f"Total Features in {district}: {total_features}")

        # Annual mean data downloading
        # if os.path.exists(os.path.join(OUTPUT_FOLDER, f"{district}_indices.csv")):
        #     print(f"Features already extracted for {district} ")
        #     continue
            
        fetch_indices_for_district(df, district, None, start_date, end_date)
        
        # Month wise data donwloading 
        # for month in date_range_month.keys():
        #     if os.path.exists(os.path.join(OUTPUT_FOLDER, f"{district}_{month.upper()}_indices.csv")):
        #         print(f"Features already extracted for {district} for month {month.upper()}")
        #         continue
        #     print(f"\nFetching data for month : {month.upper()}")
            
        #     start_date, end_date = date_range_month[month]
        #     fetch_indices_for_district(df, district, month, start_date, end_date)
            
        end_time = time.perf_counter()
        elapsed_time = end_time - start_time
        a = datetime.timedelta(seconds=elapsed_time)
        print("Time taken : " + str(a))
        

    print("\nAll districts processed successfully!")

In [ ]:
main()

Total Districts: 38

Processing District: ARARIA
Total Features in ARARIA: 1778
